# How to store transations to Embedding Model in PromptSail

In [6]:
import os
from dotenv import load_dotenv, dotenv_values
from pprint import pprint


config = dotenv_values(".env")


azure_oai_key = config["AZURE_OPENAI_API_KEY"]
api_base_url = config["AZURE_OPENAI_ENDPOINT"]

deployment_name = config["AZURE_ENDPOINT_OPENAI_DEPLOYMENT_NAME"]
api_version = config["AZURE_OPENAI_API_VERSION"]

print(
    f"Azure OpenAI api key={azure_oai_key[0:3]}...{azure_oai_key[-5:]}"
)
print(
    f"Azure OpenAI api endpoint={api_base_url[0:17]}..."
)
print(f"Azure OpenAI deployment name={deployment_name[0:7]}...")
print(f"Azure OpenAI api version={api_version}")

Azure OpenAI api key=9b2...d67b3
Azure OpenAI api endpoint=https://openai-pr...
Azure OpenAI deployment name=ada-emb...
Azure OpenAI api version=2023-07-01-preview


In [7]:

from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings


# Create an instance of the OpenAIEmbeddings class using Azure OpenAI
embeddings = AzureOpenAIEmbeddings(
    openai_api_key=azure_oai_key,
    azure_deployment=deployment_name,
    api_version=api_version,
    azure_endpoint=api_base_url,
    #model="ada-embeddings",
    chunk_size=1)

# Testing embeddings
txt = "The Ultimate Question of Life, the Universe, and Everything is 42."

# Embed a single document
e = embeddings.embed_query(txt)

print(len(e)) # should be 1536

1536


## Create a request to the AzureOpenAI Embedding API via PromptSail proxy

Run the docker and go to PromptSail UI http://promptsail.local/


You should add your own mappings in project settings. Create new project with you `project_slug`or edit existing one.

Set the `api_base` to your Azure OpenAI endpoint like
 
'https://**azure_openai_resource**.openai.azure.com/'
 
 and provider_name to Azure OpenAI.



```bash
{
    ai_providers: [
        {
            api_base: 'https://azure_openai_resource.openai.azure.com',
            provider_name: 'Azure OpenAI',
            ai_model_name: 'gpt-3.5-turbo'
        }
    ],
}
```

In this case we will use the project with project_slug -> 'project2'


In [11]:
from langchain.embeddings import AzureOpenAIEmbeddings


ps_api_base = "http://promptsail.local:8000/project2/"

# Create an instance of the OpenAIEmbeddings class using Azure OpenAI
embeddings = AzureOpenAIEmbeddings(
    openai_api_key=azure_oai_key,
    azure_deployment=deployment_name,
    api_version=api_version,
    azure_endpoint=ps_api_base,
    #model="ada-embeddings",
    chunk_size=1)

# Testing embeddings
txt = "The Ultimate Question of Life, the Universe, and Everything is 42."

# Embed a single document
e = embeddings.embed_query(txt)

print(len(e)) # should be 1536

1536
